In [1]:
RANDOM_STATE = 1

# Imports

In [2]:
import pandas as pd
import time
import numpy as np
import pickle
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth',300)

from nltk.corpus import stopwords
import re
import nltk
from nltk import tokenize
from nltk.stem import WordNetLemmatizer

import math

import spacy
import os
import pyprojroot.here as here

# Import datasets from drive

In [3]:
# TODO: modify these list if needed (eg. if you want to load only 1 csv from star3, delete other csvs in star3 list)
star3 = ['ibis-sg-bencoolen.csv','hotel-boss.csv','hotel-G.csv',
           'village-hotel-albert-court-by-far-east-hospitality.csv',
           'holiday-inn-express-clarke-quay.csv']
star4 = ['village-hotel-changi-by-far-east-hospitality.csv',
         'park-regis.csv', 'grand-mercure-sg-roxy.csv',
         'paradox-sg-merchant-court.csv','crowne-plaza.csv']
star5 = ['fullerton.csv', 'parkroyal-collection-marina-bay.csv', 'pan-pacific.csv',
          'mbs_total.csv', 'swissotel-the-stamford.csv']

RAW_FOLDER = "data/raw/"

def combine_csv_to_dataframe(file_names, all_star = False, filterDate = True):
    """
    Combine multiple CSV files into a single DataFrame.

    Parameters:
    file_names (list): List of CSV file names. 
    all_star (bool): whether or not to load all the hotels (False if only want to load 1 type of hotel star). 
    filterData (bool): whether or not to remove all data dated before 2015

    Returns:
    pd.DataFrame: Combined DataFrame.
    """
    combined_df = pd.DataFrame()

    for file_name in file_names:
        file_interim_path = RAW_FOLDER + file_name
        file_path = here(file_interim_path)
        try:
            df = pd.read_csv(file_path)
            if all_star:
                if file_name in star3:
                    df["star"] = 3
                elif file_name in star4:
                    df["star"] = 4
                else:
                    df["star"] = 5
            #print(f"Length of {file_name} is {len(df)}")
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            #print(len(combined_df))
        except FileNotFoundError:
            print(f"File not found: {file_name}")
        except pd.errors.EmptyDataError:
            print(f"Empty or invalid CSV file: {file_name}")
                                
    return combined_df

In [4]:
df = combine_csv_to_dataframe(star3+star4+star5, all_star = True, filterDate = True)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101761 entries, 0 to 101760
Data columns (total 11 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   date_of_stay                           101054 non-null  object 
 1   traveller_username                     101761 non-null  object 
 2   review_title                           101721 non-null  object 
 3   review_text                            101761 non-null  object 
 4   travel_type                            46064 non-null   object 
 5   traveller_country_origin               83686 non-null   object 
 6   traveller_total_contributions          101558 non-null  object 
 7   traveller_total_helpful_contributions  84123 non-null   float64
 8   rating1                                86074 non-null   object 
 9   rating2                                15663 non-null   float64
 10  star                                   101761 non-null  

,date_of_stay,traveller_username,review_title,review_text,travel_type,traveller_country_origin,traveller_total_contributions,traveller_total_helpful_contributions,rating1,rating2,star
0,Date of stay: August 2023,Love_Life_Sydney,Clean and comfortable,"Hotel rooms in Singapore are so expensive so to find a decent hotel, with easy transport access and food locations, at less than S$200 was a good deal. The compact room meets your basic needs, no fancy mood lightning or lovely artworks to grace the wall. It has clean sheets and towels, a small b...",Trip type: Travelled as a couple,"Sydney, Australia",2302.0,871.0,NaN,4.0,3
1,Date of stay: August 2023,Bilal S,"Good hotel, great location",This is a great place! Location is great but the room is very small. There is no room service available. Overall it's ok and they were kind enough to hold my luggage after check out! It is a good hotel!!,Trip type: Travelled with family,"Houston, Texas",4.0,NaN,5.0,NaN,3
2,Date of stay: October 2022,Anthony Fernando,Good place for a decent price.,Good place good price Easy access to the city. All walking distance. Very close to the buggies junction. Food comers around you. Also walking distance to marina bay sands Cozy rooms. Easy checking and check out. Worth for the price. Quick getaway.,Trip type: Travelled with friends,"Dubai, United Arab Emirates",39.0,38.0,5.0,NaN,3
3,Date of stay: August 2023,Mjkc204,Great Location and great staff.,"The IBIS was a neat and tidy hotel in line with the star rating. The staff on the front desk were super helpful and friendly. The hotel itself was in a great location from Orchard Road, Little India and had some local markets just behind it with the Bugis shopping malls. An atm and 7-11 was next...",Trip type: Travelled solo,"Ellenbrook, Australia",37.0,19.0,5.0,NaN,3
4,Date of stay: August 2022,Aung Nanda,Good for budget stay.,I stayed there for 7 days. It was a nice location. Seven eleven is next door. Easy access to the downtown and there was also a night market in the neighborhood. Room was nice. However I had to wait for a long time to check-in but overall experience was good.,Trip type: Travelled on business,"Dubai, United Arab Emirates",3.0,4.0,NaN,4.0,3


# Clean Data

## Clean Ratings column

In [5]:
def rating_clean(rating1, rating2):
    try:
        if not math.isnan(rating1):
            return rating1
    except:
        try:
            if not math.isnan(rating2):
                return rating2
        except:
            return None
    return rating2

def valid_rating(rating):
    try:
        if math.isnan(rating):
            return False
        return True
    except:
        return False

def classify_rating(rating):
    if rating>=4:
        return "Positive"
    if rating<=2:
        return "Negative"
    if rating == 3:
        return "Neutral"
    return None

In [6]:
df["rating"] = df.apply(lambda row: rating_clean(row['rating1'], row['rating2']), axis = 1)
df['valid_rating'] = df.apply(lambda row: valid_rating(row['rating']), axis = 1)
df["label"] = df.apply(lambda row: classify_rating(row['rating']), axis = 1)
df["label"].value_counts()

label
Positive    71398
Neutral      8465
Negative     4984
Name: count, dtype: int64

## Cleaning Raw title and review, and date

In [7]:
# combine raw title and review
def combine(r):
  return str(r['review_title']) + " " + str(r['review_text'])

In [8]:
df["combined_review"] = df.apply(lambda row: combine(row), axis = 1)
df.head(1)

,date_of_stay,traveller_username,review_title,review_text,travel_type,traveller_country_origin,traveller_total_contributions,traveller_total_helpful_contributions,rating1,rating2,star,rating,valid_rating,label,combined_review
0,Date of stay: August 2023,Love_Life_Sydney,Clean and comfortable,"Hotel rooms in Singapore are so expensive so to find a decent hotel, with easy transport access and food locations, at less than S$200 was a good deal. The compact room meets your basic needs, no fancy mood lightning or lovely artworks to grace the wall. It has clean sheets and towels, a small b...",Trip type: Travelled as a couple,"Sydney, Australia",2302.0,871.0,NaN,4.0,3,4.0,True,Positive,"Clean and comfortable Hotel rooms in Singapore are so expensive so to find a decent hotel, with easy transport access and food locations, at less than S$200 was a good deal. The compact room meets your basic needs, no fancy mood lightning or lovely artworks to grace the wall. It has clean sheets..."


## Specify Covid

In [9]:
df['date'] = df['date_of_stay'].str.split(':').str[1]
df["date"] = pd.to_datetime(df["date"])
# Extracting the year from the 'date' column
df['year'] = df['date'].dt.year

df.head(1)

,date_of_stay,traveller_username,review_title,review_text,travel_type,traveller_country_origin,traveller_total_contributions,traveller_total_helpful_contributions,rating1,rating2,star,rating,valid_rating,label,combined_review,date,year
0,Date of stay: August 2023,Love_Life_Sydney,Clean and comfortable,"Hotel rooms in Singapore are so expensive so to find a decent hotel, with easy transport access and food locations, at less than S$200 was a good deal. The compact room meets your basic needs, no fancy mood lightning or lovely artworks to grace the wall. It has clean sheets and towels, a small b...",Trip type: Travelled as a couple,"Sydney, Australia",2302.0,871.0,NaN,4.0,3,4.0,True,Positive,"Clean and comfortable Hotel rooms in Singapore are so expensive so to find a decent hotel, with easy transport access and food locations, at less than S$200 was a good deal. The compact room meets your basic needs, no fancy mood lightning or lovely artworks to grace the wall. It has clean sheets...",2023-08-01,2023.0


In [10]:
from datetime import date, timedelta, datetime

In [11]:
covid_start = datetime(2020, 1, 29, 0, 0)
covid_end = datetime(2022, 4, 1, 0, 0, 0)
def get_period(t):
    if pd.isnull(t):
        return None
    if t - covid_start < timedelta(0):
        return "PreCovid"
    elif t-covid_end >= timedelta(0):
        return "PostCovid"
    return "Covid"

In [12]:
df["covid"] = df.apply(lambda row: get_period(row["date"]), axis = 1)

In [13]:
df.head(3)

,date_of_stay,traveller_username,review_title,review_text,travel_type,traveller_country_origin,traveller_total_contributions,traveller_total_helpful_contributions,rating1,rating2,star,rating,valid_rating,label,combined_review,date,year,covid
0,Date of stay: August 2023,Love_Life_Sydney,Clean and comfortable,"Hotel rooms in Singapore are so expensive so to find a decent hotel, with easy transport access and food locations, at less than S$200 was a good deal. The compact room meets your basic needs, no fancy mood lightning or lovely artworks to grace the wall. It has clean sheets and towels, a small b...",Trip type: Travelled as a couple,"Sydney, Australia",2302.0,871.0,NaN,4.0,3,4.0,True,Positive,"Clean and comfortable Hotel rooms in Singapore are so expensive so to find a decent hotel, with easy transport access and food locations, at less than S$200 was a good deal. The compact room meets your basic needs, no fancy mood lightning or lovely artworks to grace the wall. It has clean sheets...",2023-08-01,2023.0,PostCovid
1,Date of stay: August 2023,Bilal S,"Good hotel, great location",This is a great place! Location is great but the room is very small. There is no room service available. Overall it's ok and they were kind enough to hold my luggage after check out! It is a good hotel!!,Trip type: Travelled with family,"Houston, Texas",4.0,NaN,5.0,NaN,3,5.0,True,Positive,"Good hotel, great location This is a great place! Location is great but the room is very small. There is no room service available. Overall it's ok and they were kind enough to hold my luggage after check out! It is a good hotel!!",2023-08-01,2023.0,PostCovid
2,Date of stay: October 2022,Anthony Fernando,Good place for a decent price.,Good place good price Easy access to the city. All walking distance. Very close to the buggies junction. Food comers around you. Also walking distance to marina bay sands Cozy rooms. Easy checking and check out. Worth for the price. Quick getaway.,Trip type: Travelled with friends,"Dubai, United Arab Emirates",39.0,38.0,5.0,NaN,3,5.0,True,Positive,Good place for a decent price. Good place good price Easy access to the city. All walking distance. Very close to the buggies junction. Food comers around you. Also walking distance to marina bay sands Cozy rooms. Easy checking and check out. Worth for the price. Quick getaway.,2022-10-01,2022.0,PostCovid


## Travel type

In [15]:
def short_type(t):
  try:
    return t.split(" ")[-1]
  except:
    return

In [16]:
df["travel_type"] = df.apply(lambda row: short_type(row["travel_type"]), axis = 1)
df.travel_type.value_counts()

travel_type
couple      17059
family      11984
business     9573
friends      4411
solo         3037
Name: count, dtype: int64

## Local vs Foreigner

In [23]:
# Creating a new column based on the presence of "Singapore"
df['is_local'] = df['traveller_country_origin'].apply(lambda x: 1 if 'singapore' in str(x).lower() else 0)
df.is_local.value_counts()

is_local
0    90496
1    11265
Name: count, dtype: int64

## Drop irrelevant columns

In [25]:
col_drop = ["date_of_stay", "traveller_username", "review_title", "review_text", "traveller_country_origin", "traveller_total_contributions", "traveller_total_helpful_contributions"]
df = df.drop(columns=col_drop)
df.head(1)

,travel_type,rating1,rating2,star,rating,valid_rating,label,combined_review,date,year,covid,is_local
0,couple,NaN,4.0,3,4.0,True,Positive,"Clean and comfortable Hotel rooms in Singapore are so expensive so to find a decent hotel, with easy transport access and food locations, at less than S$200 was a good deal. The compact room meets your basic needs, no fancy mood lightning or lovely artworks to grace the wall. It has clean sheets...",2023-08-01,2023.0,PostCovid,0


# Export Cleaned Data

In [47]:
def clean_data(csv):
    
    data_path = here('data/raw/' + csv)
    
    df = pd.read_csv(here(data_path))
    
    df["rating"] = df.apply(lambda row: rating_clean(row['rating1'], row['rating2']), axis = 1)
    
    df['valid_rating'] = df.apply(lambda row: valid_rating(row['rating']), axis = 1)
    
    df["label"] = df.apply(lambda row: classify_rating(row['rating']), axis = 1)
        
    df["combined_review"] = df.apply(lambda row: combine(row), axis = 1)
    
    df['date'] = df['date_of_stay'].str.split(':').str[1]
    
    df["date"] = pd.to_datetime(df["date"])
    
    df["covid"] = df.apply(lambda row: get_period(row["date"]), axis = 1)
    
    df["travel_type"] = df.apply(lambda row: short_type(row["travel_type"]), axis = 1)
    
    df['is_local'] = df['traveller_country_origin'].apply(lambda x: 1 if 'singapore' in str(x).lower() else 0)

    col_drop = ["date_of_stay", "traveller_username", "review_title", "review_text", "traveller_country_origin", 
                "traveller_total_contributions", "traveller_total_helpful_contributions",
               "rating1", "rating2"]
    
    df = df.drop(columns=col_drop)
    
    file_path = here('data/cleaned/cleaned_' + csv)
    
    
    df.to_csv(file_path, index=False)

In [48]:
for csv in star3+star4+star5:
    clean_data(csv)